In [ ]:
# flake8: noqa
import numpy as np
import scipp as sc


def solve_for_calibration_parameters(Io, I):
    Iopp, Iopa, Ioap, Ioaa = Io
    Ipp, Ipa, Iap, Iaa = I

    I0 = 2 * (Iopp * Ioaa - Iopa * Ioap) / (Iopp + Ioaa - Iopa - Ioap)

    rho = (Ioaa - Ioap) / (Iopp - Iopa)
    alp = (Ioaa - Iopa) / (Iopp - Ioap)

    Rspp_plus_Rsaa = 4 * (rho * alp * Ipp + Iaa + rho * Ipa + alp * Iap) / ((1 + rho) * (1 + alp) * I0)

    Pp = sc.sqrt(
        (Ipp + Iaa - Ipa - Iap) * (alp * (Ipp - Iap) - Iaa + Ipa) /
        ((rho * alp * Ipp + Iaa + rho * Ipa + alp * Iap) * (rho * (Ipp - Ipa) - Iaa + Iap))
    )

    Ap = sc.sqrt(
        (Ipp + Iaa - Ipa - Iap) * (rho * (Ipp - Ipa) - Iaa + Iap) /
        ((rho * alp * Ipp + Iaa + rho * Ipa + alp * Iap) * (alp * (Ipp - Iap) - Iaa + Ipa))
    )

    Rs = sc.sqrt(((alp * (Ipp - Iap) - Iaa + Ipa)) * (rho * (Ipp - Ipa) - Iaa + Iap) / ((rho * alp * Ipp + Iaa + rho * Ipa + alp * Iap) * (Ipp + Iaa - Ipa - Iap)))

    Pa = -rho * Pp
    Aa = -alp * Ap
    
    Rspp_minus_Rsaa = Rs * Rspp_plus_Rsaa
    Rspp = (Rspp_plus_Rsaa + Rspp_minus_Rsaa) / 2
    Rsaa = Rspp_plus_Rsaa - Rspp

    return I0 / 4, Pp, Pa, Ap, Aa, Rspp, Rsaa


def generate_valid_calibration_parameters():
    I0 = np.random.random()
    Pp = np.random.random()
    Pa = -np.random.random()
    Ap = np.random.random()
    Aa = -np.random.random()
    Rspp = np.random.random()
    Rsaa = Rspp * np.random.random()
    return tuple(map(sc.scalar, (I0, Pp, Pa, Ap, Aa, Rspp, Rsaa)))


def intensity_from_parameters(I0, Pp, Pa, Ap, Aa, Rpp, Rpa, Rap, Raa):
    return (
        I0 * (Rpp * (1 + Ap) * (1 + Pp) + Rpa * (1 - Ap) * (1 + Pp) + Rap * (1 + Ap) * (1 - Pp) + Raa * (1 - Ap) * (1 - Pp)),
        I0 * (Rpp * (1 + Aa) * (1 + Pp) + Rpa * (1 - Aa) * (1 + Pp) + Rap * (1 + Aa) * (1 - Pp) + Raa * (1 - Aa) * (1 - Pp)),
        I0 * (Rpp * (1 + Ap) * (1 + Pa) + Rpa * (1 - Ap) * (1 + Pa) + Rap * (1 + Ap) * (1 - Pa) + Raa * (1 - Ap) * (1 - Pa)),
        I0 * (Rpp * (1 + Aa) * (1 + Pa) + Rpa * (1 - Aa) * (1 + Pa) + Rap * (1 + Aa) * (1 - Pa) + Raa * (1 - Aa) * (1 - Pa)),
    )


## Sanity check: Can we find the polarization parameters given the measured intensities?

In [ ]:
i0, Pp, Pa, Ap, Aa, Rpp, Raa = generate_valid_calibration_parameters()

I0 = intensity_from_parameters(i0, Pp, Pa, Ap, Aa, sc.scalar(1), sc.scalar(0), sc.scalar(0), sc.scalar(1))
I = intensity_from_parameters(i0, Pp, Pa, Ap, Aa, Rpp, sc.scalar(0), sc.scalar(0), Raa)

solve_for_calibration_parameters(I0, I)

In [ ]:
# Ground truth
i0, Pp, Pa, Ap, Aa, Rpp, Raa

In [ ]:
def correction_matrix(Pp, Pa, Ap, Aa):
    return [
        [(1 + Pp) * (1 + Ap), (1 + Pp) * (1 - Ap), (1 - Pp) * (1 + Ap), (1 - Pp) * (1 - Ap)],
        [(1 + Pp) * (1 + Aa), (1 + Pp) * (1 - Aa), (1 - Pp) * (1 + Aa), (1 - Pp) * (1 - Aa)],
        [(1 + Pa) * (1 + Ap), (1 + Pa) * (1 - Ap), (1 - Pa) * (1 + Ap), (1 - Pa) * (1 - Ap)],
        [(1 + Pa) * (1 + Aa), (1 + Pa) * (1 - Aa), (1 - Pa) * (1 + Aa), (1 - Pa) * (1 - Aa)],
    ]
    

def reduce_to_q(I, q, qbins):
    if isinstance(I, list | tuple):
        return [reduce_to_q(i, q, qbins) for i in I]
    return I.assign_coords(Q=q(I)).flatten(I.dims, to='_').hist(Q=qbins)


def compute_calibration_factors(Io, I):
    I0, Pp, Pa, Ap, Aa, _, _ = solve_for_calibration_parameters(Io, I)
    return I0, correction_matrix(Pp, Pa, Ap, Aa)


def linsolve(A, b):
    An = np.stack([[a.values for a in row] for row in A])
    An = An.transpose((*range(2, An.ndim), 0, 1))
    bn = np.stack([bi.values for bi in b], axis=-1)
    xn = np.linalg.solve(An, bn)
    xn = xn.transpose((xn.ndim-1, *range(xn.ndim-1)))
    out = []
    for bi, v in zip(b, xn):
        out.append(bi.copy())
        out[-1].values[:] = v
    return out


def compute_reflectivity_alt1(Io, Is, I, q, qbins):
    'Compute factors on lz, reduce to q, apply correction on q'
    I0, C = compute_calibration_factors(Io, Is)
    return linsolve(
        reduce_to_q(
            [[I0 * c  for c in row] for row in C],
            q, qbins,
        ),
        reduce_to_q(I, q, qbins)
    )


def compute_reflectivity_alt2(Io, Is, I, q, qbins):
    'Compute factors on lz, apply correction on lz, reduce to q'
    I0, C = compute_calibration_factors(Io, Is)
    i = (
        reduce_to_q(
            linsolve(C, I),
            q, qbins
        )
    )
    r = reduce_to_q(I0, q, qbins)
    return [ii / r for ii in i]


def compute_reflectivity_alt3(Io, Is, I, q, qbins):
    'Compute factors on q, reduce to q, apply correction on q'
    I0, C = compute_calibration_factors(reduce_to_q(Io, q, qbins), reduce_to_q(Is, q, qbins))
    i = linsolve(C, reduce_to_q(I, q, qbins))
    r = I0
    return [ii / r for ii in i]



In [ ]:

def run_experiment(M, N, K, noise_level, smooth_polarizer_reflectivity, title, path):
    
    wavelength = sc.linspace('wavelength', 0.1, 1, M)
    theta = sc.linspace('z', 0.2, 0.5, N)
    q = theta / wavelength
    
    def Rs(q):
        return 0.5 * (1 + sc.cos(sc.scalar(31., unit='rad') * q / 5)) / (1 + q**2)
    
    def sharp_drop(l):
        return sc.where(l > 0.2, sc.linspace('wavelength', 0.5, 1, M), sc.scalar(0.1))
    
    def noise():
        return sc.array(dims=('wavelength', 'z'), values=noise_level * np.random.randn(M, N))
    
    i0, Pp, Pa, Ap, Aa, Rpp, Raa = (
        sc.array(dims=('wavelength', 'z'), values=np.ones((M, N))),
        sc.linspace('wavelength', 0.5, 1, M) if smooth_polarizer_reflectivity else sharp_drop(wavelength),
        -(sc.linspace('wavelength', 0.5, 1, M) if smooth_polarizer_reflectivity else sharp_drop(wavelength)),
        sc.linspace('wavelength', 0.5, 1, M) if smooth_polarizer_reflectivity else sharp_drop(wavelength),
        -(sc.linspace('wavelength', 0.5, 1, M) if smooth_polarizer_reflectivity else sharp_drop(wavelength)),
        Rs(q),
        0.5 * Rs(q),
    )
    
    *I0, = map(lambda i: sc.DataArray(i * (1 + noise())), intensity_from_parameters(
        i0, Pp, Pa, Ap, Aa,
        i0, 0*i0,
        0*i0, i0,
    ))
    *Is, = map(lambda i: sc.DataArray(i * (1 + noise())), intensity_from_parameters(
        i0, Pp, Pa, Ap, Aa,
        Rpp, Rpp*0,
        Raa*0, Raa
    ))
    *I, = map(lambda i: sc.DataArray(i * (1 + noise())), intensity_from_parameters(
        i0, Pp, Pa, Ap, Aa,
        0.5 * Rpp, 0.2 * Rpp,
        0.3 * Raa, 0.4 * Raa
    ))
    
    qf = lambda I: q
    qbins = sc.geomspace('Q', 0.2, 5, K)

    r1 = compute_reflectivity_alt1(I0, Is, I, qf, qbins)
    r2 = compute_reflectivity_alt2(I0, Is, I, qf, qbins)
    r3 = compute_reflectivity_alt3(I0, Is, I, qf, qbins)

    i = reduce_to_q(tuple(map(sc.DataArray, [0.5 * Rpp, 0.2 * Rpp, 0.3 * Raa, 0.4 * Raa])), qf, qbins)
    r = reduce_to_q(sc.DataArray(i0), qf, qbins)
    gt = [ii/r for ii in i]

    polarizer_reflectivity_features = "P_p is sharp" if not smooth_polarizer_reflectivity else "P_p is smooth"

    suffix = f'l{M}_z{N}_q{K}_eps{noise_level}_{"smooth" if smooth_polarizer_reflectivity else "sharp"}'

    methods = {1: 'Mix', 2: 'On $\lambda z$', 3: 'On $Q$'}

    p = sc.plot({
        #methods[1]: sc.abs(r1[0] - gt[0]),
        methods[2]: sc.abs(r2[0] - gt[0]),
        methods[3]: sc.abs(r3[0] - gt[0]),
    },  title=f'{title} Error, method comparison, {polarizer_reflectivity_features}')
    p.save(f'{path}/error_{suffix}.png')
    

    for m, i in (
        #(1, r1),
        (r2, 2),
        (r3, 3)
    ):
        p = sc.plot({'ground truth': gt[0], 'computed reflectivity': m[0]}, title=f'{title} $R(Q)$, method: {methods[i]}, {polarizer_reflectivity_features}')
        p.save(f'{path}/reflectivity_{i}_{suffix}.png')

    if smooth_polarizer_reflectivity:
        p = sc.DataArray(Pp).plot(title='Polarizer reflectivity - smooth')
        p.save(f'{path}/polarizer_reflectivity_smooth.png')
    else:
        p = sc.DataArray(Pp).plot(title='Polarizer reflectivity - sharp drop')
        p.save(f'{path}/polarizer_reflectivity_sharp_drop.png')

In [ ]:
run_experiment(400, 500, 400, 0, True, '', 'docs/user-guide/estia/figures')
run_experiment(400, 500, 400, 0, False, '', 'docs/user-guide/estia/figures')
run_experiment(400, 500, 400, 0.01, True, 'Realistic counts -', 'docs/user-guide/estia/figures')
run_experiment(400, 500, 400, 0.1, True, 'Low counts -', 'docs/user-guide/estia/figures')

| Property       | On $Q$       | On $\lambda z$       | Calibrate on $\lambda z$, correct on $Q$      |
|--------------|---------------|---------------|---------------|
| Can pre-reduce reference        | Yes          | Yes      | Yes          |
| Can avoid reducing reference on $\lambda z$  grid        | Yes          | No      | No          |
| Can avoid reducing sample on $\lambda z$  grid        | Yes          | No      | Yes          |
| Runtime   | Least        | Longest     | Medium       |
| Unbiased in the high counts limit   | No          | Yes          | Yes        |
| Robust to low counts  | Yes          | No        | No           |